In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ydata-profiling

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import missingno as mno
import plotly.offline as pyo 
import plotly.figure_factory as ff
import plotly.io as pio
color_pal = sns.color_palette()
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')
import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

#Model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# **EDA**

In [ ]:
df=pd.read_csv('/kaggle/input/global-university-rankings-dataset-2023/world_university_rank.csv')

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
df.describe(include = 'object').T

# **DUPLICATE**

In [ ]:
# Finding duplicate rows
duplicate_rows = df[df.duplicated(keep='first')]

# Number of duplicate rows
num_duplicates = duplicate_rows.shape[0]

# Displaying the duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
duplicate_rows

In [ ]:
df.dtypes

In [ ]:
#df['Number of Studnet'] = pd.to_numeric(df['Number of Studnet'], errors='coerce')
df['Number of Studnet'] = df['Number of Studnet'].str.replace(',', '').astype(int) 

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

# **DROP NULL**

In [ ]:
df=df.dropna()
df.shape

In [ ]:
df.head()

# **CONVERT OBJECT TO NUMERIC**

In [ ]:
#df['International Student'] = df['International Student'].str.replace('%', '', regex=True).astype(int) 
df['International Student'] = df['International Student'].str.replace('%', ' ')
df['International Student'] = pd.to_numeric(df['International Student'], errors='coerce')



In [ ]:
df['International Student']=  df['International Student']/100

In [ ]:
df.columns = df.columns.str.strip()

# Split the 'Female : Male Ratio' column and create new 'Female' and 'Male' columns
df[['Female', 'Male']] = df['Female : male ratio'].str.split(':', expand=True).astype(int)

# Drop the original 'Female : Male Ratio' column
df.drop(columns=['Female : male ratio'], inplace=True)

In [ ]:
df['locationLocation'].unique()

# **Visualization**

In [ ]:
# Import the ProfileReport class from the ydata_profiling library
from ydata_profiling import ProfileReport

# Create a comprehensive profile report for the DataFrame 'df'
# This report will contain various statistics, insights, and visualizations about the data
profile = ProfileReport(df)


In [ ]:

profile                     # or this one
profile.to_notebook_iframe() # use this line to show the output

In [ ]:
monthly_trends = df.groupby('locationLocation')['Number of Studnet'].sum().reset_index()

# Create a line plot using Plotly Express
# x-axis: Job titles, y-axis: ,
# labels for the x-axis, title, and customized height
fig_monthly_trends = px.line(
    monthly_trends,  # DataFrame containing the data
    x='locationLocation',   # x-values: job titles
    y='Number of Studnet',  # y-values: sum of salaries
    labels={'job_title': 'Job'},  # Customize label for the x-axis
    title='Number of Studnet Trends by location',  # Set the title of the plot
    height=800  # Set the height of the plot
)

# Display the plot
fig_monthly_trends.show()

In [ ]:
# Calculate the value counts for each unique value in the 'job_title' column
top10_job_title = df['locationLocation'].value_counts()[:10]

# Create a bar plot using Plotly Express
# y-axis: Count of each job title, x-axis: Job titles,
# color-coded by job titles and using a color palette
fig = px.bar(
    y=top10_job_title.values,  # y-values: count of each job title
    x=top10_job_title.index,   # x-values: job titles
    color=top10_job_title.index,  # Color the bars based on job titles
    color_discrete_sequence=px.colors.sequential.PuBuGn,  # Set color palette
    text=top10_job_title.values,  # Display the count values on top of the bars
    title='Top 10 locationL ',  # Set the title of the plot
    template='plotly_dark'  # Use a dark template for the plot
)

# Update the layout of the plot
fig.update_layout(
    title_text='location Distribution',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="location",  # Label for the x-axis
    yaxis_title="Count",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# x-axis: "company_size", y-axis: "salary_in_usd", color-coded by "experience_level"
fig = px.bar(df, x="locationLocation", y="Number of Studnet")

# Update the layout of the plot
fig.update_layout(
    title_text='Number of Studnet Distribution with location',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="location",  # Label for the x-axis
    yaxis_title="Number of Studnet",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Select the top 5 salaries in the DataFrame
top5_salary = df['Number of Studnet'].sort_values(ascending=False).head(5)

# Create a bar plot using Plotly Express
# y-axis: Salary values, x-axis: Index of the top 5 salaries,
# color-coded by the index, and using a color palette
fig = px.bar(
    y=top5_salary.values,  # y-values: top 5 salaries
    x=top5_salary.index,   # x-values: indices of the top 5 salaries
    color=top5_salary.index,  # Color the bars based on the indices
    color_discrete_sequence=px.colors.sequential.PuBuGn,  # Set color palette
    text=top5_salary.values,  # Display the salary values on top of the bars
    title='Top 5 Number of Studnet',  # Set the title of the plot
    template='plotly_dark'  # Use a dark template for the plot
)

# Update the layout of the plot
fig.update_layout(
    title_text='Number of Studnet Distribution',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="Number of Studnet",  # Label for the x-axis
    yaxis_title="Count",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Sunburst chart for the distribution of Country
fig2 = px.sunburst(df, path=['locationLocation'], color_discrete_sequence=px.colors.qualitative.Set3)
fig2.update_layout(title_text='Distribution of experience_level', height=500)
fig2.show()

In [ ]:
'''# Sunburst chart for the distribution of Country
fig2 = px.sunburst(df, path=['experience_level'], color_discrete_sequence=px.colors.qualitative.Set3)
fig2.update_layout(title_text='Distribution of experience_level', height=500)
fig2.show()'''

In [ ]:
# x-axis: "job_title", y-axis: "salary_in_usd", color-coded by "experience_level"
fig = px.scatter(df, x="locationLocation", y="Number of student per staffs")

# Update the layout of the plot
fig.update_layout(
    title_text='Number of student per staffs Distribution',  # Set the title of the plot
    height=800,  # Set the height of the plot
    xaxis_title="location",  # Label for the x-axis
    yaxis_title="Number of student per staffs",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# x-axis: "work_year", y-axis: "salary_in_usd", color-coded by "experience_level",
# and create a notched box plot
fig = px.box(df, x=df["locationLocation"]=='United States', y="International Student", notched=True)

# Update the layout of the plot
fig.update_layout(
    title_text='International Student Box Distribution',  # Set the title of the plot
    height=500,  # Set the height of the plot
    xaxis_title="location in United States",  # Label for the x-axis
    yaxis_title="International Student",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Calculate the average salary for each job title and sort in descending order
average_salary = df.groupby('locationLocation')['International Student'].mean().reset_index()
average_salary = average_salary.sort_values(by='International Student', ascending=False)

# Select the top 10 job titles with the highest average salary
top_10_expensive_salary = average_salary.head(10)

# Create a new figure using Plotly
fig = go.Figure()

# Add a bar trace to the figure
fig.add_trace(go.Bar(
    x=top_10_expensive_salary['locationLocation'],  # x-values: job titles
    y=top_10_expensive_salary['International Student']  # y-values: average salaries
))

# Update the layout and appearance of the plot
fig.update_layout(
    title='Top 10 International Student by location',  # Set the title of the plot
    xaxis_title='location',  # Label for the x-axis
    yaxis_title='International Student',  # Label for the y-axis
    paper_bgcolor='#595964',  # Background color of the plot
    template='plotly_dark',  # Use a dark template for the plot
    font=dict(color='white'),  # Set font color to white
    height=650  # Set the height of the plot
)

# Initialize Plotly for notebook integration
pyo.init_notebook_mode(connected=True)

# Display the plot
pyo.iplot(fig)

In [ ]:
import plotly.express as px
v = px.funnel(df[:20], x="International Student", y="locationLocation")
v.show()

In [ ]:
import plotly.express as px
v = px.data.gapminder()
fig = px.scatter_geo(df, locations="locationLocation",color="locationLocation",hover_name="University name")
fig.show()

# **Categorical**

In [ ]:
df['locationLocation'] = le.fit_transform(df['locationLocation'])

In [ ]:
Corr_Matrix = df.corr()

# Set up the figure and plot the heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(Corr_Matrix, annot=True, cmap='coolwarm', center=0)
plt.show()

In [ ]:
df.isna().sum()

In [ ]:
df=df.dropna()

# **TOP Positively Correlated And Negatively Correlated**

In [ ]:
print('Top 5 Most Positively Correlated to the To salary_in_usd')
Corr_Matrix['Female'].sort_values(ascending=False).head(5)

In [ ]:
print('Top 5 Most Negatively Correlated to salary_in_usd ')
Corr_Matrix['Female'].sort_values(ascending=True).head(5)

# **Split and Model** 

In [ ]:
X = df.drop(columns=['Rank','University name','Female'])
y = df['Female']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['Actual_salary'] = y_test
    submit['Predict_salary'] = y_pred
    submit = submit.reset_index()
    r2 = r2_score(y_test, y_pred)
    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(submit.head(5))

    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")

# **feature importances**

In [ ]:
importances = model.feature_importances_

feature_names = X.columns

feature_importance_dict = dict(zip(feature_names, importances))

sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance:.2f}")

plt.figure(figsize=(12, 7))
plt.barh(*zip(*sorted_feature_importance), alpha=0.9, color='teal')
plt.title('Feature Importance', fontsize=15)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

# **Neural network classification model**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assume 'features' are the columns you want to use for classification
features = df.drop(columns=['Rank','University name','Female'], axis=1)  # Replace 'target_column_name' with the actual column name

# Assume 'target' is the column you want to predict
target = df['Female']  # Replace 'target_column_name' with the actual column name

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert target labels to one-hot encoded vectors
y_train_one_hot = tf.one_hot(y_train, depth=4)
y_test_one_hot = tf.one_hot(y_test, depth=4)

# Create a classification model
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]),
    Dense(32, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train_one_hot, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test_one_hot)
print(f"Test accuracy: {accuracy:.4f}")

In [ ]:
df.dtypes